In [202]:
import numpy as np
import pandas as pd
import sklearn as sk
import convertDictionaries as cd

#import classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.svm import SVC

#import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score, recall_score, precision_score

In [203]:
data = pd.read_csv("./finalfinalDataset.csv")

In [204]:
y = pd.read_csv('./output_tidy.csv').drop('Unnamed: 0', axis=1)

In [205]:
data.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       'wnioskowania', 'wolicjonalny', 'wolitywny', 'zdarzeniowy', 'o1+',
       'o1-', 'o1o', 'o2+', 'o2-', 'o2o'],
      dtype='object', length=325)

Create test and train indices

In [206]:
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state=42).split(data, groups=data['verb']))

In [207]:
data = data.drop(["index", 'Unnamed: 0', 'verb'], axis=1)

In [208]:
y_onehot = data.iloc[:, -6:]

In [209]:
X = data.iloc[:, :-6]

In [210]:
X_train = X.iloc[train_inds] 
X_test = X.iloc[test_inds] 
y_train = y_onehot.iloc[train_inds]  
y_test = y_onehot.iloc[test_inds] 

In [211]:
estimators = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(), 
    MLPClassifier(),
    
    #te rzucaja bledem
    #AdaBoostClassifier(),
    #GaussianProcessClassifier(),
    #GradientBoostingClassifier(),
    #SVC(kernel="linear"),
    #SVC(kernel="rbf")
]

In [212]:
def check_estimator(estimator):
    
    print()
    print()
    print(str(estimator.__class__.__name__))
    
    #cross_val_score(estimator, X, y_onehot, cv=5)
    
    model = estimator.fit(X_train, y_train)
    model.score(X_test, y_test)
    
    prediction = model.predict(X_test)
    
    classNames = data.columns[-6:].to_numpy()
    precision = precision_score(y_test.to_numpy(),prediction, average="samples")
    recall = recall_score(y_test.to_numpy(),prediction, average="samples")
    
    print("class names: {0}".format(classNames))
    print("precision: {0}".format(precision))
    print("recall: {0}".format(recall))
    

In [213]:
for estimator in estimators:
    check_estimator(estimator)
    
    
    



SVC


ValueError: bad input shape (454, 6)